<a href="https://colab.research.google.com/github/yasirfaizahmed/Text-Classification-models/blob/main/models/text_sentimental_classification_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install datasets
! pip install --user -U nltk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.6 MB/s eta 0:00:00


In [2]:
from datasets import load_dataset

dataset = load_dataset("imsoumyaneel/sentiment-analysis-llama2")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

In [3]:
import pandas as pd

dataset_df = pd.DataFrame(dataset['train'])

In [4]:
dataset_df.head()

,sentence,label,text
0,I'll throw out the garbage .,neutral,###Human:\nyou are a sentiment analist. guess ...
1,"So Dick , how about getting some coffee for to...",joy,###Human:\nyou are a sentiment analist. guess ...
2,"Come on , you can at least try a little , besi...",neutral,###Human:\nyou are a sentiment analist. guess ...
3,What ’ s wrong with that ? Cigarette is the th...,anger,###Human:\nyou are a sentiment analist. guess ...
4,"Not for me , Dick .",neutral,###Human:\nyou are a sentiment analist. guess ...


In [5]:
dataset_df['label'] = dataset_df['label'].map({'joy': 0, 'neutral': 1, 'sadness': 2, 'anger': 3, 'fear': 4, 'love': 5, 'surprise': 6})
dataset_df.head()


,sentence,label,text
0,I'll throw out the garbage .,1,###Human:\nyou are a sentiment analist. guess ...
1,"So Dick , how about getting some coffee for to...",0,###Human:\nyou are a sentiment analist. guess ...
2,"Come on , you can at least try a little , besi...",1,###Human:\nyou are a sentiment analist. guess ...
3,What ’ s wrong with that ? Cigarette is the th...,3,###Human:\nyou are a sentiment analist. guess ...
4,"Not for me , Dick .",1,###Human:\nyou are a sentiment analist. guess ...


In [6]:
import nltk
from nltk.corpus import stopwords
import re
nltk.download('stopwords')
stopwords_list = stopwords.words('english')

def remove_punctuation(sentence):
  return re.sub(r'[^\w\s]','', sentence)

dataset_df['sentence'] = dataset_df['sentence'].map(remove_punctuation)


def remove_stopwords(sentence):
  new_sentence = ""
  for word in sentence.split(" "):
    if word not in stopwords_list:
      new_sentence += " " + word
  # print(new_sentence)
  return new_sentence

dataset_df['sentence'] = dataset_df['sentence'].map(remove_stopwords)
dataset_df

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


,sentence,label,text
0,Ill throw garbage,1,###Human:\nyou are a sentiment analist. guess ...
1,So Dick getting coffee tonight,0,###Human:\nyou are a sentiment analist. guess ...
2,Come least try little besides cigarette,1,###Human:\nyou are a sentiment analist. guess ...
3,What wrong Cigarette thing I go crazy,3,###Human:\nyou are a sentiment analist. guess ...
4,Not Dick,1,###Human:\nyou are a sentiment analist. guess ...
...,...,...,...
598293,You got banned participating brigade,2,###Human:\nyou are a sentiment analist. guess ...
598294,A joke subjective pal second need maturegrow ...,0,###Human:\nyou are a sentiment analist. guess ...
598295,Well Im glad youre How awful The way act make...,0,###Human:\nyou are a sentiment analist. guess ...
598296,Everyone likes NAME,5,###Human:\nyou are a sentiment analist. guess ...


In [17]:
import tensorflow as tf

tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
                                                  lower=True,
                                                  split=' ')
tokenizer.fit_on_texts(dataset_df['sentence'])
tokenized_data = tokenizer.texts_to_sequences(dataset_df['sentence'])


In [25]:
print("0th text: ", tokenizer.sequences_to_texts(tokenized_data)[0])
print("0th token: ", tokenized_data[0])
print("actual 0th text: ", dataset_df['sentence'][0])

0th text:  ill throw garbage
0th token:  [77, 1131, 2991]
actual 0th text:   Ill throw garbage 


In [30]:
max_length = 200
vocab_size = len(tokenizer.word_index)
tokenized_data = tf.keras.preprocessing.sequence.pad_sequences(tokenized_data,
                                                               maxlen=max_length,
                                                               padding='post')

array([[   77,  1131,  2991, ...,     0,     0,     0],
       [  314,  4186,    81, ...,     0,     0,     0],
       [   68,   212,   103, ...,     0,     0,     0],
       ...,
       [   35,     4,   231, ...,     0,     0,     0],
       [  151,  1472,    18, ...,     0,     0,     0],
       [   90, 20315,  1566, ...,     0,     0,     0]], dtype=int32)